In [10]:
#_filter_magics_i1
##%overwritefile
##%file:src/_filter_magics_i1.py
##%noruncode

for pkey,pvalue in self.IBplugins.items():
    # print( pkey +":"+str(len(pvalue))+"\n")
    for pobj in pvalue:
        newline=''
        try:
            if line.strip()[3:] in pobj.getIDBptag(pobj):
                newline=pobj.on_IBpCodescanning(pobj,magics,line)
                if newline=='':continue

        except Exception as e:
            pass
        finally:pass
        # if newline!=None and newline!='':
        #     actualCode += newline + '\n'
##

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_filter_magics_i1.py created successfully


In [11]:
#_filter_magics_i2
##%overwritefile
##%file:src/_filter_magics_i2.py
##%noruncode

for pkey,pvalue in self.ISplugins.items():
    # print( pkey +":"+str(len(pvalue))+"\n")
    for pobj in pvalue:
        newline=''
        try:
            if key in pobj.getIDSptag(pobj):
                newline=pobj.on_ISpCodescanning(pobj,key,value,magics,line)
                if newline=='':continue

        except Exception as e:
            pass
        finally:pass
        if newline!=None and newline!='':
            actualCode += newline + '\n'
# always add empty line, so line numbers don't change
# actualCode += '\n'

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_filter_magics_i2.py created successfully


In [12]:
#_filter_magics_pend
##%overwritefile
##%file:src/_filter_magics_pend.py
##%noruncode

if len(self.addkey2dict(magics,'file'))>0 :
    newactualCode=''
    for line in actualCode.splitlines():
        try:
            if len(self.addkey2dict(magics,'test'))<1:
                line=self.cleantestcode(line)
            if line=='':continue
            ##调用双标签接口
            line=self.callIDplugin(line)
            ##
            if line=='':continue
            line=self.cleandqm(line)
            if line=='':continue
            line=self.cleansqm(line)
            if self.cleannotes(line)=='':
                continue
            else:
                newactualCode += line + '\n'
        except Exception as e:
            self._log(str(e),3)

[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/_filter_magics_pend.py created successfully


In [14]:
#Python _filter_magics
##%overwritefile
##%file:src/py_filter_magics.py
##%noruncode
    def _filter_magics(self, code):

        magics = {
                  'repllistpid': [],
                  'replcmdmode': [],
                  'replprompt': [],
                  'replsetip': "\r\n",
                  'replchildpid':"0",

                #   'file': [],
                #   'overwritefile': [],
                #   'include': [],
                #   'templatefile': [],
                #   'test': [],

                #   'showpid': [],
                #   'norunnotecmd': [],
                #   'noruncode': [],
                #   'command': [],
                #   'pythoncmd': [],
                #   'env':None,

                  'outputtype':'text/plain',
                  'runmode': [],
                  'pid': [],
                  'pidcmd': [],
                  'args': []}

        actualCode = ''
        newactualCode = ''

        for line in code.splitlines():
            orgline=line
            line=self.forcejj2code(line)
            if line==None or line.strip()=='': continue
            if self._is_specialID(line):
                if line.strip()[3:] == "repllistpid":
                    magics['repllistpid'] += ['true']
                    self.repl_listpid()
                    continue
                elif line.strip()[3:] == "replcmdmode":
                    magics['replcmdmode'] += ['replcmdmode']
                    continue
                elif line.strip()[3:] == "replprompt":
                    magics['replprompt'] += ['replprompt']
                    continue
                else:
                    ##调用BOOL标签接口
                    ##%include:src/_filter_magics_i1.py
                    pass
                
                findObj= re.search( r':(.*)',line)
                if not findObj or len(findObj.group(0))<2:
                    continue
                key, value = line.strip()[3:].split(":", 2)
                key = key.strip().lower()

                if key == "runmode":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='real'
                elif key == "replsetip":
                    magics['replsetip'] = value
                elif key == "replchildpid":
                    magics['replchildpid'] = value
                
                
                elif key == "pidcmd":
                    magics['pidcmd'] = [value]
                    if len(magics['pidcmd'])>0:
                        findObj= value.split(",",1)
                        if findObj and len(findObj)>1:
                            pid=findObj[0]
                            cmd=findObj[1]
                            self.send_cmd(pid=pid,cmd=cmd)
                
                elif key == "outputtype":
                    magics[key]=value
                elif key == "args":
                    # Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['args'] += [argument.strip('"')]
                else:
                    pass
                    ##调用单标签接口
                    ##%include:src/_filter_magics_i2.py
            # keep lines which did not contain magics
            else:
                actualCode += line + '\n'
        newactualCode=actualCode
        ##%include:src/_filter_magics_pend.py
        return magics, newactualCode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/py_filter_magics.py created successfully


In [13]:
#C _filter_magics
##%overwritefile
##%file:src/c_filter_magics.py
##%noruncode
    def _filter_magics(self, code):

        magics = {'cflags': [],
                  'ldflags': [],
                  'dlrun': [],

                #   'file': [],                  
                #   'include': '',
                #   'templatefile': [],
                #   'test': [],

                  'repllistpid': [],
                  'replcmdmode': [],
                  'replprompt': [],
                  'replsetip': "\r\n",
                  'replchildpid':"0",

                #   'showpid': [],
                #   'norunnotecmd': [],
                #   'noruncode': [],
                #   'command': [],

                #   'onlycsfile': [],
                #   'onlyrungcc': [],
                #   'onlyruncmd': [],
                #   'showinput': [],
                #   'env':None,

                  'outputtype':'text/plain',
                  'runmode': [], #default real,interactive mode is repl
                  'pid': [],
                  'pidcmd': [],
                  'args': [],
                  'rungdb': []}

        actualCode = ''
        newactualCode = ''

        for line in code.splitlines():
            orgline=line
            line=self.forcejj2code(line)
            if line==None or line.strip()=='': continue
            if self._is_specialID(line):
                if line.strip()[3:] == "repllistpid":
                    magics['repllistpid'] += ['true']
                    self.repl_listpid()
                    continue
                elif line.strip()[3:] == "replcmdmode":
                    magics['replcmdmode'] += ['replcmdmode']
                    continue
                elif line.strip()[3:] == "replprompt":
                    magics['replprompt'] += ['replprompt']
                    continue
                elif line.strip()[3:] == "rungdb":
                    magics['rungdb'] += ['true']
                    continue
                elif line.strip()[3:] == "dlrun":
                    magics['dlrun'] += ['true']
                    continue
                else:
                    pass
                    ##调用BOOL标签接口
                    ##%include:src/_filter_magics_i1.py

                findObj= re.search( r':(.*)',line)
                if not findObj or len(findObj.group(0))<2:
                    continue
                key, value = line.strip()[3:].split(":", 2)
                key = key.strip().lower()

                if key in ['ldflags', 'cflags']:
                    for flag in value.split():
                        magics[key] += [flag]
                elif key == "runmode":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='real'
                elif key == "replsetip":
                    magics['replsetip'] = value
                elif key == "replchildpid":
                    magics['replchildpid'] = value
                
                elif key == "pidcmd":
                    magics['pidcmd'] = [value]
                    if len(magics['pidcmd'])>0:
                        findObj= value.split(",",1)
                        if findObj and len(findObj)>1:
                            pid=findObj[0]
                            cmd=findObj[1]
                            self.send_cmd(pid=pid,cmd=cmd)
                elif key == "outputtype":
                    magics[key]=value
                elif key == "args":
                    # Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['args'] += [argument.strip('"')]
                else:
                    pass
                    ##调用单标签接口
                    ##%include:src/_filter_magics_i2.py
            # keep lines which did not contain magics
            else:
                actualCode += line + '\n'
        newactualCode=actualCode
        ##%include:src/_filter_magics_pend.py
        return magics, newactualCode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/c_filter_magics.py created successfully


In [8]:
#Dart _filter_magics
##%overwritefile
##%file:src/dart_filter_magics.py
##%noruncode
    def _filter_magics(self, code):

        magics = {
                  'repllistpid': [],
                  'replcmdmode': [],
                  'replprompt': [],
                  'replsetip': "\r\n",
                  'replchildpid':"0",

                #   'file': [],
                #   'overwritefile': [],
                #   'include': [],
                #   'templatefile': [],
                #   'test': [],

                #   'showpid': [],
                #   'norunnotecmd': [],
                #   'noruncode': [],
                #   'command': [],
                #   'fluttercmd': [],
                #   'dartcmd': [],
                #   'env':None,

                  'outputtype':'text/plain',
                  'runmode': [],
                  'pid': [],
                  'pidcmd': [],
                  'args': []}

        actualCode = ''
        newactualCode = ''

        for line in code.splitlines():
            orgline=line
            line=self.forcejj2code(line)
            if line==None or line.strip()=='': continue
            if self._is_specialID(line):
                if line.strip()[3:] == "repllistpid":
                    magics['repllistpid'] += ['true']
                    self.repl_listpid()
                    continue
                elif line.strip()[3:] == "replcmdmode":
                    magics['replcmdmode'] += ['replcmdmode']
                    continue
                elif line.strip()[3:] == "replprompt":
                    magics['replprompt'] += ['replprompt']
                    continue
                else:
                    ##调用BOOL标签接口
                    ##%include:src/_filter_magics_i1.py
                    pass

                findObj= re.search( r':(.*)',line)
                if not findObj or len(findObj.group(0))<2:
                    continue
                key, value = line.strip()[3:].split(":", 2)
                key = key.strip().lower()

                if key == "runmode":
                    if len(value)>0:
                        magics[key] = value[re.search(r'[^/]',value).start():]
                    else:
                        magics[key] ='real'
                elif key == "replsetip":
                    magics['replsetip'] = value
                elif key == "replchildpid":
                    magics['replchildpid'] = value['command'],env=magics['env'])
                
                elif key == "pidcmd":
                    magics['pidcmd'] = [value]
                    if len(magics['pidcmd'])>0:
                        findObj= value.split(",",1)
                        if findObj and len(findObj)>1:
                            pid=findObj[0]
                            cmd=findObj[1]
                            self.send_cmd(pid=pid,cmd=cmd)
                elif key == "outputtype":
                    magics[key]=value
                elif key == "args":
                    # Split arguments respecting quotes
                    for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
                        magics['args'] += [argument.strip('"')]
                else:
                    pass
                    ##调用单标签接口
                    ##%include:src/_filter_magics_i2.py
            # keep lines which did not contain magics
            else:
                actualCode += line + '\n'
        newactualCode=actualCode
        ##%include:src/_filter_magics_pend.py
        return magics, newactualCode


[MyPythonkernel] Info:file /root/Jupyter/Myjupyter-kernel/src/dart_filter_magics.py created successfully
